# GaN/Fe

## import & prepare

In [1]:
!pip install git+https://github.com/nmdl-mizo/interface_master.git@develop
!mkdir -p cif_files
!curl https://raw.githubusercontent.com/nmdl-mizo/interface_master/develop/test_files/cif_files/GaN_mp-804_conventional_standard.cif -o 'cif_files/GaN_mp-804_conventional_standard.cif'
!curl https://raw.githubusercontent.com/nmdl-mizo/interface_master/develop/test_files/cif_files/Fe_mp-13_conventional_standard.cif -o 'cif_files/Fe_mp-13_conventional_standard.cif'

DEPRECATION: Loading egg at /opt/anaconda3/envs/general/lib/python3.11/site-packages/cnidcal-0.0.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/envs/general/lib/python3.11/site-packages/interfacemaster-1.1.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Cloning https://github.com/nmdl-mizo/interface_master.git (to revision develop) to /private/var/folders/km/tznxn_d162bgzpqpzp9wcjn00000gn/T/pip-req-build-jinisnj8
  Running command git clone --filter=blob:none --quiet https://github.com/nmdl-mizo/interface_master.git /private/var/folders/km/tznxn_d162bgzpqpzp9wcjn00000gn/T/pip-req-build-jinisnj8
  Resolved https://github.com/nmdl-miz

In [1]:
from interfacemaster.cellcalc import get_primitive_hkl, get_pri_vec_inplane, get_normal_index, get_normal_from_MI, rot
from interfacemaster.interface_generator import core, convert_vector_index, get_disorientation
from numpy import array, dot, round, cross, eye
from numpy.linalg import inv, det, norm
from pymatgen.core.structure import Structure

## Define an interface core class

In [2]:
stct_1, stct_2 = list(map(lambda x:Structure.from_file(x), ['cif_files/GaN_mp-804_conventional_standard.cif', \
                                                      'cif_files/Fe_mp-13_conventional_standard.cif']))

/opt/anaconda3/envs/general/lib/python3.11/site-packages/pymatgen/io/cif.py:1229: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [3]:
my_interface = core(stct_1, stct_2)

Warning!, this programme will rewrite the POSCAR file in this dir!


In [4]:
#target disorientation
R = get_disorientation(L1 = my_interface.conv_lattice_1, L2 = my_interface.conv_lattice_2, \
                       v1 = [-1,-1,0], hkl1 = [1, -1, 0], \
                       v2 = [0,-1,0], hkl2 = [2, 0, 5])
R

array([[ 3.21633760e-01,  5.00000000e-01,  8.04084401e-01],
       [-1.85695338e-01,  8.66025404e-01, -4.64238345e-01],
       [-9.28476691e-01,  4.28707594e-17,  3.71390676e-01]])

In [5]:
#combining two specified surfaces
hkl_1 = get_primitive_hkl([1, -1, 0], my_interface.conv_lattice_1, my_interface.lattice_1)
hkl_2 = get_primitive_hkl([2, 0, 5], my_interface.conv_lattice_2, my_interface.lattice_2)

## Searching two_dimensional CSL

In [6]:
my_interface.parse_limit(du = 1e-1, S  = 1e-1, sgm1=200, sgm2=200, dd = 1e-1)

#Do searching!
my_interface.search_one_position_2D(hkl_1, hkl_2, theta_range = 1, \
                                    dtheta = 0.01, pre_R=R, pre_dt= True, integer_tol = 1e-4)

Congrates, we found an appx CSL!

U1 = 
[[-4  0]
 [-4  0]
 [ 0 -3]]; sigma_1 = 17

U2 = 
[[-5  5]
 [ 0  7]
 [-5  2]]; sigma_2 = 52

D = 
[[ 0.97649518 -0.04071154  0.        ]
 [-0.04071154  0.92948554  0.        ]
 [-0.         -0.          1.02783677]]

axis = [14.59533203 -8.42661887  0.        ] ; theta = 0.0



## Get bicrystal

In [7]:
my_interface.compute_bicrystal_two_D(hkl_1 = hkl_1, hkl_2=hkl_2, normal_ortho = True, lim = 50, tol_ortho = 1e-2, tol_integer=1e-3)

cell 1:
[[-1 -4  0]
 [ 1 -4  0]
 [ 0  0  3]]
cell 2:
[[-1 -5 -5]
 [ 1  0 -7]
 [ 2 -5 -2]]


In [8]:
my_interface.get_bicrystal(two_D = True, xyz_1 = [1,1,1], xyz_2 = [1,1,1])

Please check the POSCAR file